In [7]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

In [8]:
# Load data
df = pd.read_csv(r'D:\ta_project\data\raw\data.csv')
df.head(10)

,No,Model Pakaian,Jenis Bahan,Waktu Pengerjaan,Pengalaman Penjahit,Detail Tambahan,Harga Jasa
0,1,Kebaya Tradisional,Katun,Reguler,Pemula,Tanpa ornamen,250000
1,2,Kebaya Tradisional,Katun,Ekspres,Menengah,Bordir,300000
2,3,Kebaya Tradisional,Sutra,Reguler,Mahir,Bordir,950000
3,4,Kebaya Tradisional,Lace,Ekspres,Pemula,Payet,350000
4,5,Kebaya Tradisional,Sifon,Reguler,Menengah,Payet,450000
5,6,Kebaya Modern,Katun,Ekspres,Mahir,Tanpa ornamen,350000
6,7,Kebaya Modern,Sutra,Reguler,Pemula,Bordir,320000
7,8,Kebaya Modern,Lace,Ekspres,Menengah,Payet,380000
8,9,Kebaya Modern,Sifon,Reguler,Mahir,Payet,400000
9,10,Kebaya Modern,Satin,Ekspres,Pemula,Renda,370000


In [9]:
# Exploratory Data Analysis
print(df.shape)
print(df.describe())

(150, 7)
               No      Harga Jasa
count  150.000000      150.000000
mean    75.500000   463800.000000
std     43.445368   199632.783686
min      1.000000   250000.000000
25%     38.250000   330000.000000
50%     75.500000   395000.000000
75%    112.750000   480000.000000
max    150.000000  1000000.000000
